## Weather information at random location with KNMI hourly weather API

This project uses the KNMI Hourly weather information to determine the weather at a random location at a random day and hour.
The weather information for the Netherlands is requested from the [KNMI API](https://www.knmi.nl/kennis-en-datacentrum/achtergrond/data-ophalen-vanuit-een-script). and the random locations are determined for The Netherlands.

### 1. Determine a random date random, call the KNMI weather data API and store the weather and weather stations information

In [123]:
"""
Created on Wed Aug  31

@author: kraaitim
"""
import random
import requests
from io import StringIO
import pandas as pd

random_start_day = random.randint(1, 10)  # Random start day
random_end_day = random.randint(random_start_day, 30)  # Random end day
# Random start month between January and March
random_start_month = random.randint(5, 7)
# Random month between start month and August
random_end_month = random.randint(random_start_month, 8)
random_year = 2022
print(
    f'start date: {random_year}{str(random_start_month).zfill(2)}{str(random_start_day).zfill(2)}')
print(
    f'end date: {random_year}{str(random_end_month).zfill(2)}{str(random_end_day).zfill(2)}')

# API call parameters
params = {
    # Random start date in format YYYYMMDD
    "start": f'{random_year}{str(random_start_month).zfill(2)}{str(random_start_day).zfill(2)}',
    # Random end date in format YYYYMMDD
    "end": f'{random_year}{str(random_end_month).zfill(2)}{str(random_end_day).zfill(2)}',
    "vars": "T:M:R:S:O:Y",  # Temperatuur, Mist, Regen, Sneeuw, Onweer, Ijsvorming
    "snts": "ALL"  # All 49 locations
    # "fmt": "json" #Output JSON
}
# String for the API call
items = '&'.join(f'{key}={value}' for key, value in params.items())

# POST API call to KNMI
# https://www.knmi.nl/kennis-en-datacentrum/achtergrond/data-ophalen-vanuit-een-script
response = requests.post(
    'https://www.daggegevens.knmi.nl/klimatologie/uurgegevens', data=items)

# Decode bytes to string
decoded_response = response.content.decode()
# Create a text buffer and read the text into a pandas DataFrame
weather_data = pd.read_csv(StringIO(decoded_response), sep=",", comment="#", skiprows=49, header=None, names=[
                           "id", "YYYYMMDD", "Hour", "T", "M", "R", "S", "O", "Y"])

# Extract location information
locs = []
# Loop over the 6th till the 6+49location = 55th row
for loc in decoded_response.splitlines()[6:56]:
    # Extract the location id, lon, lat
    locs.append([loc[2:5], loc[14:19], loc[26:32]])

# Pandas DataFrame with column names
locations = pd.DataFrame(locs, columns=["id", "long", "lat"])

start date: 20220701
end date: 20220729


### 2. Determine a random location in The Netherlands and find the closest weather station

In [72]:
import geopy.distance
import numpy as np

# Bounding box The Netherlands
min_NL_long = 3.3425257166
max_NL_long = 7.4331117122
min_NL_lat = 50.7064973345
max_NL_lat = 53.4897071803

# Random location in the Netherlands
random_NL_long = random.uniform(min_NL_long, max_NL_long)
random_NL_lat = random.uniform(min_NL_lat, max_NL_lat)

# Random coords tuple
random_coords = (random_NL_lat, random_NL_long)
print(random_coords)

# Calculate distance in km between 2 coord tuples (lat, long)
min = np.inf
min_loc = 0

# Loop over the rows of the locations to find the weather station closest to the random location
for index, row in locations.iterrows():
    distance = geopy.distance.geodesic(random_coords, (row.lat, row.long)).km
    if distance < min:
        min_loc = row.id
        min = distance

print("Distance (km) to closest weather station:", min)
print("Closest location:", min_loc)


(52.26218836876165, 6.0654786855496)
Distance (km) to closest weather station: 23.316773814396022
Closest location: 278


### 3. Find the weather information at a random day and hour on the random location

In [146]:
random_hour = random.randint(1, 24)
random_day = random.randint(random_start_day, random_end_day)
random_month = random.randint(random_start_month, random_end_month)
random_year = 2022

#Create random date in the format of the weather data 
random_date = f'{random_year}{str(random_month).zfill(2)}{str(random_day).zfill(2)}'

print(
    f'start date: {random_year}{str(random_start_month).zfill(2)}{str(random_start_day).zfill(2)}')
print(
    f'end date: {random_year}{str(random_end_month).zfill(2)}{str(random_end_day).zfill(2)}')

print("Random date:", random_date)
print("Random Hour:", random_hour)

# Filter the dataframe by the closest weather station, year, month, date and hour.
result = weather_data[(weather_data.id == int(min_loc)) & (weather_data.YYYYMMDD == int(random_date)) & (weather_data.Hour == random_hour)]
result


start date: 20220701
end date: 20220729
Random date: 20220722
Random Hour: 21


,id,YYYYMMDD,Hour,T,M,R,S,O,Y
13052,278,20220722,21,142,,,,,
